In [2]:
import pandas as pd
#!conda install pytorch torchvision torchaudio -c pytorch --yes
# Load the data
file_path = '/home/yui/Downloads/QNN-test/pjm_east_2024_synthetic.csv'
data = pd.read_csv(file_path)

In [3]:
# Feature target split
target = data['value']
features = data.drop('value', axis=1)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
# Good old train test split
# Convert features and target to tensors
X = torch.tensor(features.values, dtype=torch.float32)
y = torch.tensor(target.values, dtype=torch.float32).unsqueeze(1)  # Ensure y is the correct shape
# Initialize the scaler
scaler = StandardScaler()
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the scaler on your TRAINING data only
X_train = scaler.fit_transform(X_train.numpy())  # Convert to NumPy array to fit
X_test = scaler.transform(X_test.numpy())  # Apply the same transform to the test data

# Convert scaled features back to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

# Create DataLoader for training
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


val_dataset = TensorDataset(X_test, y_test)

In [5]:
import torch.nn as nn
import torch.optim as optim

class QuantileNetwork(nn.Module):
    def __init__(self, input_size, num_quantiles):
        super(QuantileNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_quantiles)
        )

    def forward(self, x):
        return self.network(x)

def quantile_loss(preds, target, quantiles):
    assert len(quantiles) == preds.shape[1], "Quantiles size must match predictions width."
    errors = target.unsqueeze(1) - preds
    return torch.max((quantiles - 1) * errors, quantiles * errors).mean()

# Initialize the model
num_features = X_train.shape[1]
num_quantiles = 50
quantiles = torch.linspace(0.01, 0.99, steps=num_quantiles)
model = QuantileNetwork(input_size=num_features, num_quantiles=num_quantiles)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        preds = model(batch_x)
        loss = quantile_loss(preds, batch_y, quantiles)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 6107223040.0
Epoch 2, Loss: 5959763968.0
Epoch 3, Loss: 6639678976.0
Epoch 4, Loss: 6632449024.0
Epoch 5, Loss: 6164440576.0
Epoch 6, Loss: 6239736320.0
Epoch 7, Loss: 6447383040.0
Epoch 8, Loss: 5980840448.0
Epoch 9, Loss: 5850449920.0
Epoch 10, Loss: 6090036224.0
Epoch 11, Loss: 6178769920.0
Epoch 12, Loss: 6312083968.0
Epoch 13, Loss: 6012160000.0
Epoch 14, Loss: 6531036160.0
Epoch 15, Loss: 5982267392.0
Epoch 16, Loss: 6337232896.0
Epoch 17, Loss: 6204045312.0
Epoch 18, Loss: 6371545088.0
Epoch 19, Loss: 6026193920.0
Epoch 20, Loss: 6213103104.0
Epoch 21, Loss: 6172084736.0
Epoch 22, Loss: 6259755520.0
Epoch 23, Loss: 6323193344.0
Epoch 24, Loss: 5906226176.0
Epoch 25, Loss: 6454215168.0
Epoch 26, Loss: 6191072768.0
Epoch 27, Loss: 6149027840.0
Epoch 28, Loss: 6036398592.0
Epoch 29, Loss: 6285844992.0
Epoch 30, Loss: 6244517376.0
Epoch 31, Loss: 6196805632.0
Epoch 32, Loss: 6431005696.0


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
num_features = X_train.shape[1]
num_quantiles = 50
quantiles = torch.linspace(0.01, 0.99, steps=num_quantiles)
def quantile_loss(preds, target, quantiles):
    assert len(quantiles) == preds.shape[1], "Quantiles size must match predictions width."
    errors = target.unsqueeze(1) - preds
    return torch.max((quantiles - 1) * errors, quantiles * errors).mean()
# Define Quantile Network with three hidden layers
class QuantileNetwork(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes, dropout_rate):
        super(QuantileNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_sizes)):
            if i == 0:
                layers.append(nn.Linear(input_size, hidden_sizes[i]))
            else:
                layers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))

        layers.append(nn.Linear(hidden_sizes[-1], output_size))  # Output layer
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Train and evaluate model
def train_and_evaluate(model, train_loader, val_loader, optimizer, quantiles, epochs=2000):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = quantile_loss(output, target, quantiles)
            loss.backward()
            optimizer.step()

    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            loss = quantile_loss(output, target, quantiles)
            total_loss += loss.item()
    avg_loss = total_loss / len(val_loader)
    return avg_loss

# Hyperparameter search space adjustments
neurons_options = [
    (32, 64, 128),
    (64, 128, 256),
    (128, 256, 512)
]
batch_sizes = [128, 512]#[64, 128, 256, 512]
neurons = [32, 64, 128, 256]
optimizers_dict = {'Adam': optim.Adam, 'Adagrad': optim.Adagrad, 'RMSprop': optim.RMSprop}
dropout_rates = np.linspace(0, 0.3, num=4)
learning_rates = np.logspace(-5, -1, num=5)
# Calculate the total number of iterations
total_iterations = len(batch_sizes) * len(neurons_options) * len(dropout_rates) * len(optimizers_dict) * len(learning_rates)
current_iteration = 0
# Hyperparameter search
best_loss = np.inf
best_config = None
best_model=None
for batch_size in batch_sizes:
    for hidden_sizes in neurons_options:  # Corrected from neurons to neurons_options
        for dropout_rate in dropout_rates:
            for optimizer_name, optimizer_class in optimizers_dict.items():
                for lr in learning_rates:
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
                    # Correct instantiation with the actual input size
                    model = QuantileNetwork(input_size=85, output_size=50, hidden_sizes=hidden_sizes, dropout_rate=dropout_rate)
                    optimizer = optimizer_class(model.parameters(), lr=lr)  # Corrected from opt to optimizer
                    loss = train_and_evaluate(model, train_loader, val_loader, optimizer, quantiles, epochs=100)  # Specified epochs for clarity
                    if loss < best_loss:
                        best_loss = loss
                        best_config = (batch_size, hidden_sizes, dropout_rate, optimizer_name, lr)
                        best_model = model
                        print(f"Best Configuration So far: {best_config} with Loss: {best_loss}")
                        print(f"Search done for {batch_size},{hidden_sizes},{dropout_rate},{optimizer_name},{lr}")
                    current_iteration += 1
                    completion_percentage = (current_iteration / total_iterations) * 100
                    print(f"Completed: {completion_percentage:.2f}%")

print(f"Best Configuration: {best_config} with Loss: {best_loss}")